# State Monad

In [26]:
type Env a b = a -> Maybe b

data Exp = Num Int
         | Exp :+: Exp
         | Exp :/: Exp
         | Var String
         | Let String Int Exp
         
type ExpEnv = Env String Int

insert :: Eq a => a -> b -> Env a b -> Env a b
insert k v env = \ k' -> if k == k' then Just v else env k'

Line 12: Redundant lambda
Found:
insert k v env = \ k' -> if k == k' then Just v else env k'
Why not:
insert k v env k' = if k == k' then Just v else env k'

In [27]:
newtype State s a = St (s -> (a,s))

-- get liefert immer das s von dem state aber nur im monad kontext do oder bind
getState :: State s s
getState = St $ \s -> (s,s)

putState :: s -> State s ()
putState ns = St $ const ((), ns)

runState :: s -> State s a -> a
runState init (St sf) = fst $ sf init

In [28]:
instance Applicative (State s) where
  pure = return
  --(<*>) :: State s (a -> b) -> State s a -> State s b
  St sf <*> St sa = St $ \s ->
     let (f,s1) = sf s
         (x,s2) = sa s1 in
       (f x, s2)
  
instance Functor (State s) where
  fmap f (St sf) = St $ \s ->
     let (x,s1) = sf s in
        (f x, s1)
  
instance Monad (State s) where
  -- return :: a -> State s a
  return x = St (\s -> (x,s))
  -- (>>=) :: State s a -> (a -> State s b) -> State s b
  st_sf >>= f = St $ \s -> 
     let St sf = st_sf
         (x, s1) = sf s
         St sg = f x in
       sg s1


Line 16: Use tuple-section
Found:
\ s -> (x, s)
Why not:
(x,)

In [29]:
eval :: ExpEnv -> Exp -> Maybe Int
eval env (Num n)     = pure n
eval env (Var i)     = env i
eval env (e1 :+: e2) = (+) <$> eval env e1 <*> eval env e2
eval env (e1 :/: e2) = eval env e2 >>= \e' ->
  case e' of
    0 -> Nothing
    _ -> div <$> eval env e1 <*> pure e'

In [38]:
-- import Control.Monad.State
-- {-# LANGUAGE LambdaCase #-}

evalState :: Exp -> State ExpEnv (Maybe Int)
evalState (Num n) = pure (Just n)
evalState (Var i) = getState >>= \env -> return (env i)
--                          ExpEnv
evalState (Let s i exp) = getState >>= \env -> putState (insert s i env) >> evalState exp
evalState (e1 :+: e2) = evalState e1 >>= \res1 -> evalState e2 >>= \res2 -> return  $ (+) <$> res1 <*> res2
evalState (e1 :/: e2) = evalState e2 >>= \case 
                                    Just 0 -> return Nothing
                                    e2' -> evalState e1 >>= \e1' -> return $ div <$> e1' <*> e2'

In [ ]:
-- instance Show ExpEnv where
--     show :: ExpEnv -> String
--     show f = 